# Interactive QA Bot Documentation

This documentation will guide you through using the interactive QA bot application built with Streamlit. The application allows you to upload documents, interact with a chatbot, and view responses based on the content of the uploaded document.

## Uploading Files

1. **File Uploader**

    - **Interface**: The file uploader is located in the center of the application interface.
    - **Supported Formats**: You can upload documents in TXT or PDF format.

    ```python
    uploaded_file = st.file_uploader("Upload your document (TXT or PDF only)", type=["txt", "pdf"], help="Drag and drop file here or click to browse.")
    ```

    - **Action**: Drag and drop your file into the uploader area or click to browse for the file.

2. **Processing the Uploaded File**

    - **Temporary Storage**: The uploaded file is saved temporarily on the server.

    ```python
    temp_file_path = save_uploaded_file(uploaded_file)
    ```

    - **Initialization**: The QA bot is initialized with the path of the uploaded file.

    ```python
    chatbot = QAChatbot(temp_file_path)
    ```

    - **Success Message**: A success message will be shown once the file is uploaded and the bot is ready to use.

    ```python
    st.success("File uploaded successfully! You can now chat with the bot.")
    ```

## Interacting with the Bot

1. **Chat History Display**

    - **View Previous Messages**: The chat history displays previous interactions with the bot.

    ```python
    for message in st.session_state['messages']:
        if message['role'] == 'user':
            st.markdown(f'<div style="background-color: #e1ffc7; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>You:</strong> {message["content"]}</div>', unsafe_allow_html=True)
        else:
            st.markdown(f'<div style="background-color: #dbe6f4; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>Bot:</strong> {message["content"]}</div>', unsafe_allow_html=True)
            if 'segments' in message:
                st.markdown('<div style="background-color: #f4f4f4; color: #000; border-radius: 10px; padding: 10px; margin-top: 5px;"><strong>Retrieved Segments:</strong></div>', unsafe_allow_html=True)
                for segment in message['segments']:
                    st.markdown(f'<div style="background-color: #f9f9f9; color: #000; border-radius: 5px; padding: 5px; margin-bottom: 3px;">- {segment}</div>', unsafe_allow_html=True)
    ```

    - **User Messages**: Displayed with a light green background.
    - **Bot Responses**: Displayed with a light blue background.
    - **Retrieved Segments**: If applicable, shows the segments of the document used to generate the bot's response.

2. **Sending a Query**

    - **Input Form**: Type your query into the text input field and submit it.

    ```python
    with st.form(key='input_form', clear_on_submit=True):
        user_query = st.text_input("Type your message:", key="user_input_form")
        submit_button = st.form_submit_button("Send")
    ```

    - **Submit Query**: When you submit a query, it is added to the chat history, and the bot processes the query.

    ```python
    if submit_button and user_query:
        # Store the user query in chat history
        st.session_state['messages'].append({'role': 'user', 'content': user_query})

        # Get the response from the chatbot
        response, retrieved_segments = chatbot.interact_with_llm(user_query)

        # Store the bot's response and retrieved segments in chat history
        st.session_state['messages'].append({'role': 'bot', 'content': response, 'segments': retrieved_segments})

        # Rerun the app to update the chat history
        st.experimental_rerun()
    ```

    - **Response Handling**: The bot's response and the relevant document segments are displayed in the chat history.

## No File Uploaded

- **Warning Message**: If no file is uploaded, a warning message will prompt you to upload a TXT or PDF file to start interacting with the bot.

    ```python
    else:
        st.warning("Please upload a TXT or PDF file to start.")
    ```

This documentation should help users understand how to interact with the QA bot application, upload files, and view the responses effectively.


In [ ]:
%pip install streamlit
%pip install llama_index
%pip install llama-index-vector-stores-chroma
%pip install llama_index-embeddings-gemini
%pip install llama-index-llms-gemini
%pip install PyPDF2
%pip install chromadb
%pip install google-generativeai

In [7]:
import streamlit as st
import tempfile
import os

# Import the QAChatbot class from your backend
from Backend import QAChatbot

### Function to Save Uploaded File Temporarily

This function is responsible for temporarily saving the file uploaded by the user. When a user uploads a document, the file needs to be processed by the backend, but first, it must be saved on the system. This function handles that by saving the uploaded file in a temporary location.

1. **`uploadedfile`**: This is the file object passed when a user uploads a document (e.g., a `.txt` or `.pdf` file).
2. **`tempfile.NamedTemporaryFile`**: This creates a temporary file in the system. The `delete=False` argument ensures that the file will not be deleted automatically when closed, and `suffix='.' + uploadedfile.name.split('.')[-1]` sets the file's extension (e.g., `.txt` or `.pdf`).
3. **`uploadedfile.getbuffer()`**: Reads the file contents and stores them in a buffer so that it can be written to the temporary file.
4. **Return**: The function returns the path to the saved temporary file so that it can be processed further by the QA chatbot.

#### Why is this needed?
The file is saved temporarily so that it can be safely processed by the backend. This method ensures that the file is isolated in a temporary directory and won't interfere with other files, while also giving flexibility to handle different file types.


In [11]:
# Function to save uploaded file temporarily
def save_uploaded_file(uploadedfile):
    # Create a temporary file with the same extension as the uploaded file
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.' + uploadedfile.name.split('.')[-1])

    # Write the contents of the uploaded file into the temporary file
    temp_file.write(uploadedfile.getbuffer())

    # Return the path of the saved temporary file
    return temp_file.name


### Initializing Session State and Setting Up the Streamlit App

1. **Initialize Session State**

    ```python
    # Initialize session state to store chat history
    if 'messages' not in st.session_state:
        st.session_state['messages'] = []
    ```

    - **Purpose**: This code block initializes the session state to store the chat history.
    - **`st.session_state`**: A special dictionary-like object provided by Streamlit to store data across different user interactions and app reruns.
    - **`'messages'`**: A key used to store the chat history. It is initialized as an empty list if it does not already exist in the session state. This list will keep track of all user and bot messages throughout the session.

2. **Streamlit App Layout**

    ```python
    # Streamlit app layout
    st.set_page_config(page_title="Chat-style QA Bot", layout="wide", page_icon="💬")
    ```

    - **Purpose**: Configures the layout and appearance of the Streamlit app.
    - **`st.set_page_config`**: A function that sets the title, layout, and icon for the app.
      - **`page_title="Chat-style QA Bot"`**: Sets the title of the web page that appears in the browser tab.
      - **`layout="wide"`**: Configures the app layout to be wide, making use of the full width of the screen for better display of content.
      - **`page_icon="💬"`**: Sets the icon shown in the browser tab to a chat bubble emoji.

3. **App Header**

    ```python
    # App header
    st.markdown("<h1 style='text-align: center; color: #FF4B4B;'>💬 Chat-style QA Bot</h1>", unsafe_allow_html=True)
    st.markdown("<h4 style='text-align: center;'>Upload your document and start chatting!</h4>", unsafe_allow_html=True)
    ```

    - **Purpose**: Displays the header and subtitle of the app.
    - **`st.markdown`**: Renders Markdown and HTML content.
      - **`<h1>`**: Creates a main heading with a chat bubble emoji, centered and styled with a red color.
      - **`<h4>`**: Adds a smaller heading below the main one, instructing users to upload a document and start chatting.
      - **`unsafe_allow_html=True`**: Allows the use of raw HTML in Markdown, enabling custom styling and formatting.

4. **File Uploader**

    ```python
    # File uploader
    uploaded_file = st.file_uploader("Upload your document (TXT or PDF only)", type=["txt", "pdf"], help="Drag and drop file here or click to browse.")
    ```

    - **Purpose**: Provides a file upload widget for users to upload documents.
    - **`st.file_uploader`**: A Streamlit widget that allows users to upload files.
      - **`"Upload your document (TXT or PDF only)"`**: The label displayed to users for the upload widget.
      - **`type=["txt", "pdf"]`**: Restricts the file types that can be uploaded to text files (`.txt`) and PDF files (`.pdf`).
      - **`help="Drag and drop file here or click to browse."`**: Provides additional guidance to users on how to upload a file.


In [9]:
# Initialize session state to store chat history
if 'messages' not in st.session_state:
    st.session_state['messages'] = []

# Streamlit app layout
st.set_page_config(page_title="Chat-style QA Bot", layout="wide", page_icon="💬")

# App header
st.markdown("<h1 style='text-align: center; color: #FF4B4B;'>💬 Chat-style QA Bot</h1>", unsafe_allow_html=True)
st.markdown("<h4 style='text-align: center;'>Upload your document and start chatting!</h4>", unsafe_allow_html=True)

# File uploader
uploaded_file = st.file_uploader("Upload your document (TXT or PDF only)", type=["txt", "pdf"], help="Drag and drop file here or click to browse.")

2024-09-18 12:32:46.265 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 12:32:46.268 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-09-18 12:32:46.270 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 12:32:46.272 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 12:32:46.345 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 12:32:46.346 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

### Handling Uploaded Files and Chat Interactions

1. **Check if a File is Uploaded**

    ```python
    if uploaded_file is not None:
    ```

    - **Purpose**: This condition checks if a file has been uploaded by the user.
    - **`uploaded_file`**: Contains the file object if a file is uploaded, otherwise it is `None`.

2. **Save the Uploaded File Temporarily**

    ```python
    # Save the uploaded file temporarily
    temp_file_path = save_uploaded_file(uploaded_file)
    ```

    - **Purpose**: Calls the `save_uploaded_file` function to save the uploaded file to a temporary location.
    - **`temp_file_path`**: Stores the path to the temporarily saved file, which is used for further processing by the QA bot.

3. **Initialize the QA Bot**

    ```python
    # Initialize the QA bot with the document path
    chatbot = QAChatbot(temp_file_path)
    ```

    - **Purpose**: Creates an instance of the `QAChatbot` class, passing the path of the temporary file to it.
    - **`chatbot`**: An object of `QAChatbot` that will interact with the uploaded document and process queries.

4. **Show Success Message**

    ```python
    st.success("File uploaded successfully! You can now chat with the bot.")
    ```

    - **Purpose**: Displays a success message to the user indicating that the file has been uploaded and the chat functionality is now available.

5. **Display Chat History**

    ```python
    # Display chat
    st.markdown("### Chat History")
    ```

    - **Purpose**: Adds a heading to the page to display the chat history.

6. **Render Chat History**

    ```python
    # Chat history display
    for message in st.session_state['messages']:
        if message['role'] == 'user':
            st.markdown(f'<div style="background-color: #e1ffc7; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>You:</strong> {message["content"]}</div>', unsafe_allow_html=True)
        else:
            st.markdown(f'<div style="background-color: #dbe6f4; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>Bot:</strong> {message["content"]}</div>', unsafe_allow_html=True)
            if 'segments' in message:
                st.markdown('<div style="background-color: #f4f4f4; color: #000; border-radius: 10px; padding: 10px; margin-top: 5px;"><strong>Retrieved Segments:</strong></div>', unsafe_allow_html=True)
                for segment in message['segments']:
                    st.markdown(f'<div style="background-color: #f9f9f9; color: #000; border-radius: 5px; padding: 5px; margin-bottom: 3px;">- {segment}</div>', unsafe_allow_html=True)
    ```

    - **Purpose**: Loops through the chat history stored in `st.session_state['messages']` and renders each message on the page.
    - **User Messages**: Displayed with a light green background.
    - **Bot Responses**: Displayed with a light blue background.
    - **Retrieved Segments**: If available, shows segments from the document that were used to generate the bot's response.

7. **User Input for Chat**

    ```python
    # User input for chat
    with st.form(key='input_form', clear_on_submit=True):
        user_query = st.text_input("Type your message:", key="user_input_form")
        submit_button = st.form_submit_button("Send")
    ```

    - **Purpose**: Provides a form for the user to type and submit their query.
    - **`st.form`**: Creates a form with a key to manage state and clear input upon submission.
    - **`st.text_input`**: A text input field for the user to type their message.
    - **`st.form_submit_button`**: A button to submit the form and send the query to the bot.

8. **Handle Form Submission**

    ```python
    if submit_button and user_query:
        # Store the user query in chat history
        st.session_state['messages'].append({'role': 'user', 'content': user_query})

        # Get the response from the chatbot
        response, retrieved_segments = chatbot.interact_with_llm(user_query)

        # Store the bot's response and retrieved segments in chat history
        st.session_state['messages'].append({'role': 'bot', 'content': response, 'segments': retrieved_segments})

        # Rerun the app to update the chat history
        st.experimental_rerun()
    ```

    - **Purpose**: Processes the user query when the form is submitted.
    - **Check Submission**: Verifies if the submit button was clicked and if the user query is not empty.
    - **Store Query**: Adds the user's query to the chat history.
    - **Get Response**: Calls the `interact_with_llm` method of the `chatbot` to get a response and the segments used.
    - **Store Response**: Adds the bot's response and segments to the chat history.
    - **Update UI**: Uses `st.experimental_rerun()` to refresh the app and display the updated chat history.

9. **Display Warning if No File is Uploaded**

    ```python
    else:
        st.warning("Please upload a TXT or PDF file to start.")
    ```

    - **Purpose**: Displays a warning message if no file has been uploaded, prompting the user to upload a file to interact with the bot.


In [10]:
if uploaded_file is not None:
    # Save the uploaded file temporarily
    temp_file_path = save_uploaded_file(uploaded_file)

    # Initialize the QA bot with the document path
    chatbot = QAChatbot(temp_file_path)

    st.success("File uploaded successfully! You can now chat with the bot.")

    # Display chat
    st.markdown("### Chat History")

    # Chat history display
    for message in st.session_state['messages']:
        if message['role'] == 'user':
            st.markdown(f'<div style="background-color: #e1ffc7; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>You:</strong> {message["content"]}</div>', unsafe_allow_html=True)
        else:
            st.markdown(f'<div style="background-color: #dbe6f4; color: #000; border-radius: 10px; padding: 10px; margin-bottom: 5px;"><strong>Bot:</strong> {message["content"]}</div>', unsafe_allow_html=True)
            if 'segments' in message:
                st.markdown('<div style="background-color: #f4f4f4; color: #000; border-radius: 10px; padding: 10px; margin-top: 5px;"><strong>Retrieved Segments:</strong></div>', unsafe_allow_html=True)
                for segment in message['segments']:
                    st.markdown(f'<div style="background-color: #f9f9f9; color: #000; border-radius: 5px; padding: 5px; margin-bottom: 3px;">- {segment}</div>', unsafe_allow_html=True)

    # User input for chat
    with st.form(key='input_form', clear_on_submit=True):
        user_query = st.text_input("Type your message:", key="user_input_form")
        submit_button = st.form_submit_button("Send")

        if submit_button and user_query:
            # Store the user query in chat history
            st.session_state['messages'].append({'role': 'user', 'content': user_query})

            # Get the response from the chatbot
            response, retrieved_segments = chatbot.interact_with_llm(user_query)

            # Store the bot's response and retrieved segments in chat history
            st.session_state['messages'].append({'role': 'bot', 'content': response, 'segments': retrieved_segments})

            # Rerun the app to update the chat history
            st.experimental_rerun()

else:
    st.warning("Please upload a TXT or PDF file to start.")


2024-09-18 12:32:50.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-18 12:32:50.183 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Challenges faced, and solutions
---

## Key Decisions and Design

### 1. **Choosing Streamlit for the Frontend**
- **Decision**: I selected **Streamlit** for the frontend to create a simple and interactive web-based user interface (UI).
- **Reason**: Streamlit is lightweight, easy to use, and excellent for quickly deploying AI-based applications with minimal configuration.
- **Challenge**: Integrating a smooth chat-based interface with real-time interaction and file upload functionality.
- **Solution**: Streamlit’s form-based interaction allowed for smooth handling of user inputs and dynamic UI updates. I utilized session state to manage chat history across interactions.

### 2. **Managing File Uploads**
- **Decision**: I implemented a file uploader component to allow users to upload PDF or TXT files.
- **Reason**: The chatbot needed to process documents, so an easy-to-use file upload mechanism was essential.
- **Challenge**: Handling uploaded files properly in a temporary environment and limiting supported file types.
- **Solution**: Using `st.file_uploader`, I limited file types and used the `tempfile` module for temporary file storage.

### 3. **Storing and Displaying Chat History**
- **Decision**: I used `Streamlit’s session state` to store conversation history, maintaining chat continuity between queries.
- **Reason**: Users should see previous interactions, and the bot should retain context throughout the conversation.
- **Challenge**: Ensuring the chat history persists across interactions while keeping a clean UI.
- **Solution**: `st.session_state` stored messages with roles (user/bot). Markdown formatting was used for a clean and interactive display of chat history.

### 4. **Interactive Chat Interface**
- **Decision**: Designed the chat interface to visually distinguish between user inputs, bot responses, and document segments.
- **Reason**: A clear, visually engaging chat interface improves usability and engagement.
- **Challenge**: Formatting dynamic chat history while keeping document segments separate.
- **Solution**: Applied different colors for user (`#e1ffc7`) and bot (`#dbe6f4`) messages using HTML/CSS via Markdown.

### 5. **Form for User Input**
- **Decision**: Used `st.form` to handle user input for controlled submission of chat updates.
- **Reason**: A form allows controlled submission, easier validation, and state management.
- **Challenge**: Ensuring responsive, real-time chat input processing.
- **Solution**: `st.form_submit_button` efficiently handled user input and triggered backend processing, updating chat history dynamically.

### 6. **Handling Empty States**
- **Decision**: Added a warning message when no file was uploaded to guide users.
- **Reason**: Providing clear instructions enhances the user experience.
- **Challenge**: Preventing user interaction without necessary inputs and providing feedback.
- **Solution**: Displayed `st.warning` when no file was uploaded, with additional instructions in the `help` parameter.

---

## Challenges and Solutions

### 1. **Maintaining Chat Continuity**
- **Challenge**: Streamlit resets state after each interaction, making conversation history tricky to maintain.
- **Solution**: Utilized `st.session_state` to store and update the chat history with each interaction.

### 2. **Displaying Document Segments**
- **Challenge**: Displaying both bot responses and retrieved document segments without cluttering the interface.
- **Solution**: Designed a visually distinct section for document segments, maintaining clarity in the chat interface.

### 3. **Responsive Layout**
- **Challenge**: Ensuring the layout adapts to different screen sizes, especially smaller screens.
- **Solution**: Used `st.set_page_config` to set the layout to `wide`, optimizing for different devices.

### 4. **Handling Temporary Files**
- **Challenge**: Managing uploaded files efficiently in a temporary environment.
- **Solution**: Leveraged Python’s `tempfile` module to store files temporarily during the session.

### 5. **Ensuring Clear User Feedback**
- **Challenge**: Providing clear feedback on user actions (file upload, queries) to enhance user experience.
- **Solution**: Added success and warning messages using `st.success` and `st.warning` to guide users.

---

## Key Components

### 1. **File Upload Handling**
- The `save_uploaded_file()` function saves the uploaded document temporarily for backend processing.

### 2. **Session State for Chat History**
- Chat history is maintained in `st.session_state`, ensuring continuity across interactions.

### 3. **User Input via Form**
- User input is managed via a form, ensuring controlled and validated submission of queries.

### 4. **Dynamic Chat Rendering**
- Chat history is rendered using Markdown, providing a clean, interactive display for both user and bot messages.

### 5. **Real-Time Interaction**
- The backend’s `interact_with_llm()` method processes user queries in real time, updating chat history dynamically.

---

This explanation outlines the major design decisions, challenges, and solutions implemented in the `app.py` code, ensuring a smooth and user-friendly chatbot experience through Streamlit.


---

## Key Decisions and Design

### 1. **Choosing Streamlit for the Frontend**
- **Decision**: I selected **Streamlit** for the frontend to create a simple and interactive web-based user interface (UI).
- **Reason**: Streamlit is lightweight, easy to use, and excellent for quickly deploying AI-based applications with minimal configuration.
- **Challenge**: Integrating a smooth chat-based interface with real-time interaction and file upload functionality.
- **Solution**: Streamlit’s form-based interaction allowed for smooth handling of user inputs and dynamic UI updates. I utilized session state to manage chat history across interactions.

### 2. **Managing File Uploads**
- **Decision**: I implemented a file uploader component to allow users to upload PDF or TXT files.
- **Reason**: The chatbot needed to process documents, so an easy-to-use file upload mechanism was essential.
- **Challenge**: Handling uploaded files properly in a temporary environment and limiting supported file types.
- **Solution**: Using `st.file_uploader`, I limited file types and used the `tempfile` module for temporary file storage.

### 3. **Storing and Displaying Chat History**
- **Decision**: I used `Streamlit’s session state` to store conversation history, maintaining chat continuity between queries.
- **Reason**: Users should see previous interactions, and the bot should retain context throughout the conversation.
- **Challenge**: Ensuring the chat history persists across interactions while keeping a clean UI.
- **Solution**: `st.session_state` stored messages with roles (user/bot). Markdown formatting was used for a clean and interactive display of chat history.

### 4. **Interactive Chat Interface**
- **Decision**: Designed the chat interface to visually distinguish between user inputs, bot responses, and document segments.
- **Reason**: A clear, visually engaging chat interface improves usability and engagement.
- **Challenge**: Formatting dynamic chat history while keeping document segments separate.
- **Solution**: Applied different colors for user (`#e1ffc7`) and bot (`#dbe6f4`) messages using HTML/CSS via Markdown.

### 5. **Form for User Input**
- **Decision**: Used `st.form` to handle user input for controlled submission of chat updates.
- **Reason**: A form allows controlled submission, easier validation, and state management.
- **Challenge**: Ensuring responsive, real-time chat input processing.
- **Solution**: `st.form_submit_button` efficiently handled user input and triggered backend processing, updating chat history dynamically.

### 6. **Handling Empty States**
- **Decision**: Added a warning message when no file was uploaded to guide users.
- **Reason**: Providing clear instructions enhances the user experience.
- **Challenge**: Preventing user interaction without necessary inputs and providing feedback.
- **Solution**: Displayed `st.warning` when no file was uploaded, with additional instructions in the `help` parameter.

---

## Challenges and Solutions

### 1. **Maintaining Chat Continuity**
- **Challenge**: Streamlit resets state after each interaction, making conversation history tricky to maintain.
- **Solution**: Utilized `st.session_state` to store and update the chat history with each interaction.

### 2. **Displaying Document Segments**
- **Challenge**: Displaying both bot responses and retrieved document segments without cluttering the interface.
- **Solution**: Designed a visually distinct section for document segments, maintaining clarity in the chat interface.

### 3. **Responsive Layout**
- **Challenge**: Ensuring the layout adapts to different screen sizes, especially smaller screens.
- **Solution**: Used `st.set_page_config` to set the layout to `wide`, optimizing for different devices.

### 4. **Handling Temporary Files**
- **Challenge**: Managing uploaded files efficiently in a temporary environment.
- **Solution**: Leveraged Python’s `tempfile` module to store files temporarily during the session.

### 5. **Ensuring Clear User Feedback**
- **Challenge**: Providing clear feedback on user actions (file upload, queries) to enhance user experience.
- **Solution**: Added success and warning messages using `st.success` and `st.warning` to guide users.

---

## Key Components

### 1. **File Upload Handling**
- The `save_uploaded_file()` function saves the uploaded document temporarily for backend processing.

### 2. **Session State for Chat History**
- Chat history is maintained in `st.session_state`, ensuring continuity across interactions.

### 3. **User Input via Form**
- User input is managed via a form, ensuring controlled and validated submission of queries.

### 4. **Dynamic Chat Rendering**
- Chat history is rendered using Markdown, providing a clean, interactive display for both user and bot messages.

### 5. **Real-Time Interaction**
- The backend’s `interact_with_llm()` method processes user queries in real time, updating chat history dynamically.

---
